### ЗАВДАННЯ 2

In [3]:
class Matrix:

    def __init__(self, lst):
        try:
            for i in range(len(lst)):
                if type(lst) == list and type(lst[i]) == list:       
                    if lst == [[]]:
                        self.matrix = [[]]
                        return None
                    for x in lst[i]:
                        if not (type(x) == int or type(x) == float):
                            print("all elements of matrix must be numbers")
                            self.matrix = [[]]
                            return None
                    self.matrix = lst                               
                else:                                                
                    print("lst isn't list of lists")
                    self.matrix = [[]]
                    return None
        except TypeError:                                            
            print("lst isn't list of lists")
            self.matrix = [[]]

            
    def __str__(self):
        return self.matrix
    
    
    def change_lines(self, line_1, line_2):  
        if not (type(line_1) == int and type(line_2) == int and line_1 <= len(self.matrix)-1 and line_2 <= len(self.matrix)-1):
            print("Uncorrect numbers of lines!")
            return None
        self.matrix[line_1], self.matrix[line_2] = self.matrix[line_2], self.matrix[line_1]
           
            
            
    def multi_line(self, line, digit):    
        if not (type(line) == int and type(digit) == int and line <= len(self.matrix)-1 and digit != 0):
            print("Uncorrect numbers of lines")
            return None
        self.matrix[line] = [i * digit for i in self.matrix[line]]    
        
        
    def division_line(self, line, digit):   
        if not (type(line) == int and type(digit) == int and line <= len(self.matrix)-1 and digit != 0):
            print("Uncorrect numbers of lines")
            return None
        self.matrix[line] = [i // digit for i in self.matrix[line]]     
   

    def add_line(self, line_1, line_2, digit): 
        if not (type(line_1) == int and type(line_2) == int and type(digit) == int and line_1 <= len(self.matrix)-1 and line_2 <= len(self.matrix)-1 and digit != 0):
            print("Uncorrect numbers of lines")
            return None
        added_line = [i * digit for i in self.matrix[line_2]]
        for i in range(len(self.matrix[line_1])):
            self.matrix[line_1][i] += added_line[i]   
    
    

Використовуючи елементарні перетворення рядків, перетворити матрицю Test в верхню трикутну матрицю:

In [4]:
list_1 = [[4, 2, 0], [1, 3, 2], [-1, 3, 10]]
Test = Matrix(list_1)
print(Test.__str__())    

[[4, 2, 0], [1, 3, 2], [-1, 3, 10]]


Поміняємо перший та другий рядок місцями:

In [5]:
Test.change_lines(0, 1)
print(Test.__str__())  

[[1, 3, 2], [4, 2, 0], [-1, 3, 10]]


До 2-го рядка додати 1-й, помножений на -4:

In [6]:
Test.add_line(1, 0, -4)
print(Test.__str__())  

[[1, 3, 2], [0, -10, -8], [-1, 3, 10]]


До 3-го рядка додати 1-й:

In [7]:
Test.add_line(2, 0, 1)
print(Test.__str__())  

[[1, 3, 2], [0, -10, -8], [0, 6, 12]]


2-й рядок поділити на -2:

In [8]:
Test.division_line(1, -2)
print(Test.__str__())  

[[1, 3, 2], [0, 5, 4], [0, 6, 12]]


3-й рядок ділимо на 6:

In [9]:
Test.division_line(2, 6)
print(Test.__str__())  

[[1, 3, 2], [0, 5, 4], [0, 1, 2]]


Поміняємо 2-й та 3-й рядок місцями:

In [10]:
Test.change_lines(1, 2)
print(Test.__str__())  

[[1, 3, 2], [0, 1, 2], [0, 5, 4]]


До 3-го рядка додамо 2-й, помножений на -5:

In [11]:
Test.add_line(2, 1, -5)
print(Test.__str__())  

[[1, 3, 2], [0, 1, 2], [0, 0, -6]]
